In [ ]:
import pandas as pd
import re
from openpyxl import Workbook

# Helper function to convert month numbers to names
def month_number_to_name(month):
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

# Helper function to determine the last three months
def get_last_three_months(current_month, current_year):
    months = []
    for i in range(3):
        month = (current_month - i - 1) % 12 + 1
        year = current_year if current_month - i > 0 else current_year - 1
        months.append((month, year))
    return months

# Function to process MoM data
def process_mom_data(sheet, sheet_name, current_month, current_year, data):
    last_three_months = get_last_three_months(current_month, current_year)
    
    for index, row in sheet.iterrows():
        entity = row.get('Entity', '')
        function = row.get('RTN Level 4', '')
        mica = row.get('Account', '')

        for month_num, year in last_three_months:
            month_col = f"M{month_num:02d} {year}"
            if month_col in sheet.columns:
                period_value = f"{month_number_to_name(month_num)}-{str(year)[-2:]}"
                if 'DC' in sheet_name:
                    data.append([sheet_name, '', '', '', '', '', 'MoM', period_value, row[month_col], '', '', '', '', entity, function, '', mica])
                elif 'HC' in sheet_name:
                    data.append([sheet_name, '', '', '', '', '', 'MoM', period_value, '', row[month_col], '', '', '', entity, function, '', mica])

# Function to calculate YTD data
def process_ytd_data(sheet, sheet_name, current_month, current_year, data):
    for index, row in sheet.iterrows():
        entity = row.get('Entity', '')
        function = row.get('RTN Level 4', '')
        mica = row.get('Account', '')

        # Calculate YTD value
        ytd_value = sum(
            row[f'M{month:02d} {current_year}'] for month in range(1, current_month + 1)
            if f'M{month:02d} {current_year}' in sheet.columns
        )
        
        period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
        if 'DC' in sheet_name:
            data.append([sheet_name, '', '', '', '', '', 'YTD Actual', period_value, ytd_value, '', '', '', '', entity, function, '', mica])
        elif 'HC' in sheet_name:
            data.append([sheet_name, '', '', '', '', '', 'YTD Actual', period_value, '', ytd_value, '', '', '', entity, function, '', mica])

# Function to process Prior Year FY data
def process_prior_year_fy_data(sheet, current_year, data):
    prior_year = current_year - 1
    fy_col = f"FY {prior_year}"
    
    for index, row in sheet.iterrows():
        if fy_col in sheet.columns:
            entity = row.get('Entity', '')
            function = row.get('RTN Level 4', '')
            mica = row.get('Account', '')
            if 'DC' in sheet_name:
                data.append([sheet_name, '', '', '', '', '', 'Prior Year FY', '', row[fy_col], '', '', '', '', entity, function, '', mica])
            elif 'HC' in sheet_name:
                data.append([sheet_name, '', '', '', '', '', 'Prior Year FY', '', '', row[fy_col], '', '', '', entity, function, '', mica])

# Function to process Prior Year YTD data
def process_prior_year_ytd_data(sheet, current_month, current_year, data):
    prior_year = current_year - 1
    ytd_value = sum(
        row[f'M{month:02d} {prior_year}'] for month in range(1, current_month + 1)
        if f'M{month:02d} {prior_year}' in sheet.columns
    )

    period_value = f"{month_number_to_name(current_month)}-{str(prior_year)[-2:]}"
    entity = row.get('Entity', '')
    function = row.get('RTN Level 4', '')
    mica = row.get('Account', '')
    
    if 'DC' in sheet_name:
        data.append([sheet_name, '', '', '', '', '', 'Prior Year YTD', period_value, ytd_value, '', '', '', '', entity, function, '', mica])
    elif 'HC' in sheet_name:
        data.append([sheet_name, '', '', '', '', '', 'Prior Year YTD', period_value, '', ytd_value, '', '', '', entity, function, '', mica])

# Setup
current_month = 1  # Example: January
current_year = 2025
processed_file_path = 'Processed_file.xlsx'

xls = pd.ExcelFile(processed_file_path)
wb = Workbook()
ws = wb.active
ws.title = "Paste"
headers = ["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
           "Finance Region", "Attribute Type", "Period", "Cost", 
           "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
           "Function", "Mapped Country", "MICA"]
ws.append(headers)

data = []

# Process sheets
for sheet_name in xls.sheet_names:
    sheet = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Process MoM
    if 'Actual' in sheet_name:
        process_mom_data(sheet, sheet_name, current_month, current_year, data)
    
    # Process YTD
    if 'YTD' in sheet_name:
        process_ytd_data(sheet, sheet_name, current_month, current_year, data)

    # Process Prior Year FY
    if 'FY' in sheet_name:
        process_prior_year_fy_data(sheet, current_year, data)

    # Process Prior Year YTD
    if 'YTD' in sheet_name:
        process_prior_year_ytd_data(sheet, current_month, current_year, data)

# Write final data to the worksheet
for row in data:
    ws.append(row)

# Save the final workbook
wb.save('final_output_file.xlsx')


In [ ]:
import pandas as pd
from openpyxl import Workbook

# Helper function to convert month numbers to names
def month_number_to_name(month):
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

# Helper function to get the last three months
def get_last_three_months(current_month, current_year):
    months = []
    for i in range(3):
        month = (current_month - i - 1) % 12 + 1
        year = current_year if current_month - i > 0 else current_year - 1
        months.append((month, year))
    return months

# Function to process MoM data
def process_mom_data(sheet, sheet_name, current_month, current_year):
    last_three_months = get_last_three_months(current_month, current_year)
    mom_data = []

    for index, row in sheet.iterrows():
        entity = row.get('Entity', '')
        function = row.get('RTN Level 4', '')
        mica = row.get('Account', '')

        for month_num, year in last_three_months:
            month_col = f"M{month_num:02d} {year}"
            if month_col in sheet.columns:
                period_value = f"{month_number_to_name(month_num)}-{str(year)[-2:]}"
                if 'DC' in sheet_name.upper():
                    mom_data.append([sheet_name, '', '', '', '', '', 'MoM', period_value, row[month_col], '', '', '', '', entity, function, '', mica])
                elif 'HC' in sheet_name.upper():
                    mom_data.append([sheet_name, '', '', '', '', '', 'MoM', period_value, '', row[month_col], '', '', '', entity, function, '', mica])

    return pd.DataFrame(mom_data)

# Function to process YTD data
def process_ytd_data(sheet, sheet_name, current_month, current_year):
    ytd_data = []

    for index, row in sheet.iterrows():
        entity = row.get('Entity', '')
        function = row.get('RTN Level 4', '')
        mica = row.get('Account', '')

        total_ytd = sum(row[f"M{month:02d} {current_year}"] for month in range(1, current_month + 1) if f"M{month:02d} {current_year}" in sheet.columns)

        if total_ytd:
            period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
            if 'DC' in sheet_name.upper():
                ytd_data.append([sheet_name, '', '', '', '', '', 'YTD Actual', period_value, total_ytd, '', '', '', '', entity, function, '', mica])
            elif 'HC' in sheet_name.upper():
                ytd_data.append([sheet_name, '', '', '', '', '', 'YTD Actual', period_value, '', total_ytd, '', '', '', entity, function, '', mica])

    return pd.DataFrame(ytd_data)

# Function to process Prior Year FY data
def process_prior_year_fy_data(sheet, sheet_name, current_year):
    prior_year_fy_data = []
    prior_year_fy_col = f"FY {current_year - 1}"

    if prior_year_fy_col not in sheet.columns:
        return pd.DataFrame()

    for index, row in sheet.iterrows():
        entity = row.get('Entity', '')
        function = row.get('RTN Level 4', '')
        mica = row.get('Account', '')

        if not pd.isna(row[prior_year_fy_col]):
            period_value = f"Dec-{str(current_year - 1)[-2:]}"
            if 'DC' in sheet_name.upper():
                prior_year_fy_data.append([sheet_name, '', '', '', '', '', 'Prior Year FY', period_value, row[prior_year_fy_col], '', '', '', '', entity, function, '', mica])
            elif 'HC' in sheet_name.upper():
                prior_year_fy_data.append([sheet_name, '', '', '', '', '', 'Prior Year FY', period_value, '', row[prior_year_fy_col], '', '', '', entity, function, '', mica])

    return pd.DataFrame(prior_year_fy_data)

# Function to process Prior Year YTD data
def process_prior_year_ytd_data(sheet, sheet_name, current_month, current_year):
    prior_year_ytd_data = []

    for index, row in sheet.iterrows():
        entity = row.get('Entity', '')
        function = row.get('RTN Level 4', '')
        mica = row.get('Account', '')

        total_prior_year_ytd = sum(row[f"YTD{month:02d} {current_year - 1}"] for month in range(1, current_month + 1) if f"YTD{month:02d} {current_year - 1}" in sheet.columns)

        if total_prior_year_ytd:
            period_value = f"{month_number_to_name(current_month)}-{str(current_year - 1)[-2:]}"
            if 'DC' in sheet_name.upper():
                prior_year_ytd_data.append([sheet_name, '', '', '', '', '', 'Prior Year YTD', period_value, total_prior_year_ytd, '', '', '', '', entity, function, '', mica])
            elif 'HC' in sheet_name.upper():
                prior_year_ytd_data.append([sheet_name, '', '', '', '', '', 'Prior Year YTD', period_value, '', total_prior_year_ytd, '', '', '', entity, function, '', mica])

    return pd.DataFrame(prior_year_ytd_data)

# Main function to process the workbook
def process_workbook(processed_file_path, current_month, current_year):
    xls = pd.ExcelFile(processed_file_path)

    # Create a new workbook and the "Paste" sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Paste"

    headers = ["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
               "Finance Region", "Attribute Type", "Period", "Cost", 
               "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
               "Function", "Mapped Country", "MICA"]
    ws.append(headers)

    final_df = pd.DataFrame()

    for sheet_name in xls.sheet_names:
        sheet = pd.read_excel(xls, sheet_name=sheet_name)

        # Check for relevant data and append
        if 'ACTUAL' in sheet_name.upper():
            final_df = pd.concat([final_df, process_mom_data(sheet, sheet_name, current_month, current_year)])
            final_df = pd.concat([final_df, process_ytd_data(sheet, sheet_name, current_month, current_year)])

        if 'DC' in sheet_name.upper() or 'HC' in sheet_name.upper():
            final_df = pd.concat([final_df, process_prior_year_fy_data(sheet, sheet_name, current_year)])
            final_df = pd.concat([final_df, process_prior_year_ytd_data(sheet, sheet_name, current_month, current_year)])

    for row in final_df.itertuples(index=False):
        ws.append(row)

    wb.save('final_output_file.xlsx')

# Example usage
current_month = 1  # For January
current_year = 2025
processed_file_path = 'Processed_file.xlsx'  # Update with your file path

process_workbook(processed_file_path, current_month, current_year)


In [ ]:
`

In [ ]:
import pandas as pd
import re

def month_number_to_name(month):
    """Convert month number to abbreviated month name."""
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

def process_paste_sheet(processed_file_path, current_month, current_year):
    # Load the processed file
    xls = pd.ExcelFile(processed_file_path)
    
    # Create an empty DataFrame to collect all data for the "Paste" sheet
    paste_df = pd.DataFrame(columns=["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
                                     "Finance Region", "Attribute Type", "Period", "Costimport pandas as pd
from openpyxl import Workbook

def month_number_to_name(month):
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

def add_mom(sheet, current_month, current_year, ws):
    """Handle MoM data for Actual sheets."""
    month_columns = [f"M{month:02d} {current_year}" for month in
                     [(current_month - i) % 12 or 12 for i in range(3)]]
    
    for index, row in sheet.iterrows():
        entity = row.get('Entity', '')
        function = row.get('RTN Level 4', '')
        mica = row.get('Account', '')
        
        for month_col in month_columns:
            if month_col in sheet.columns:
                value = row[month_col]
                period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
                ws.append([sheet.name, '', '', '', '', '', 'MoM', period_value, value, '', '', '', '', entity, function, '', mica])

def add_fy_target(sheet, current_month, current_year, ws):
    """Handle FY Target sheets."""
    fy_column_name = f'FY {current_year}'
    for index, row in sheet.iterrows():
        if fy_column_name in sheet.columns:
            entity = row.get('Entity', '')
            function = row.get('RTN Level 4', '')
            mica = row.get('Account', '')
            value = row[fy_column_name]

            # Determine if DC or HC
            if 'DC' in sheet.name:
                ws.append([sheet.name, '', '', '', '', '', 'FY Target', f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}", value, '', '', '', '', entity, function, '', mica])
            elif 'HC' in sheet.name:
                ws.append([sheet.name, '', '', '', '', '', 'FY Target', f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}", '', value, '', '', '', entity, function, '', mica])

def add_fy_forecast(sheet, current_year, ws):
    """Handle FY Forecast sheets."""
    fy_column_name = f'FY {current_year}'
    for index, row in sheet.iterrows():
        if fy_column_name in sheet.columns:
            entity = row.get('Entity', '')
            function = row.get('RTN Level 4', '')
            mica = row.get('Account', '')
            value = row[fy_column_name]

            # Paste data based on DC or HC
            if 'DC' in sheet.name:
                ws.append([sheet.name, '', '', '', '', '', 'FY Forecast', f"Dec-{str(current_year)[-2:]}", value, '', '', '', '', entity, function, '', mica])
            elif 'HC' in sheet.name:
                ws.append([sheet.name, '', '', '', '', '', 'FY Forecast', f"Dec-{str(current_year)[-2:]}", '', value, '', '', '', entity, function, '', mica])

def process_sheets(processed_file_path, current_month, current_year):
    # Load the processed file
    xls = pd.ExcelFile(processed_file_path)
    
    # Create a new workbook and the "Paste" sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Paste"

    # Define headers for the "Paste" sheet
    headers = ["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
               "Finance Region", "Attribute Type", "Period", "Cost", 
               "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
               "Function", "Mapped Country", "MICA"]
    ws.append(headers)  # Write headers to the first row

    for sheet_name in xls.sheet_names:
        sheet = pd.read_excel(xls, sheet_name=sheet_name)

        # Handle MoM for Actual sheets
        if 'Actual' in sheet_name:
            add_mom(sheet, current_month, current_year, ws)

        # Handle FY Target sheets
        if 'Target' in sheet_name:
            add_fy_target(sheet, current_month, current_year, ws)

        # Handle FY Forecast sheets
        if 'Fcst' in sheet_name:
            add_fy_forecast(sheet, current_year, ws)

    # Save the final workbook
    wb.save('final_output_file.xlsx')

# Example usage
current_month = 6  # June
current_year = 2024  # Current year
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
process_sheets(processed_file_path, current_month, current_year)
", 
                                     "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
                                     "Function", "Mapped Country", "MICA"])

    # Process each sheet
    for sheet_name in xls.sheet_names:
        # Only process sheets containing 'Actual', 'Target', or 'Fcst'
        if re.search(r'Actual|Target|Fcst', sheet_name, re.IGNORECASE):
            # Only load required columns from the sheet
            sheet = pd.read_excel(xls, sheet_name=sheet_name, usecols=lambda x: re.match(r'Entity|RTN Level 4|Account|M\d{2}|FY|YTD', x))

            # Ensure relevant columns exist
            if not {'Entity', 'RTN Level 4', 'Account'}.issubset(sheet.columns):
                continue

            # Extract relevant columns
            entity_col = sheet['Entity']
            function_col = sheet['RTN Level 4']
            account_col = sheet['Account']

            # Identify cost type (DC for Cost, HC for FTE/Contractor)
            cost_type = 'Cost' if 'DC' in sheet_name else 'FTE/Contractor'

            # 1. MoM (Last 3 Months Data)
            month_cols = [col for col in sheet.columns if re.match(r'M\d{2}', col)]
            for month_col in month_cols:
                month_num = int(month_col[1:3])
                period_value = f"{month_number_to_name(month_num)}-{str(current_year)[-2:]}"

                if (current_month - month_num) % 12 in [0, 1, 2]:
                    paste_df = pd.concat([paste_df, pd.DataFrame({
                        'File Name': sheet_name,
                        'Entity': entity_col,
                        'Function': function_col,
                        'Account': account_col,
                        'Attribute Type': 'MoM',
                        'Period': period_value,
                        cost_type: sheet[month_col]
                    })], ignore_index=True)

            # 2. YTD (Summing up values up to current month)
            ytd_cols = [col for col in sheet.columns if re.match(r'M\d{2}', col) and int(col[1:3]) <= current_month]
            if ytd_cols:
                sheet['YTD Total'] = sheet[ytd_cols].sum(axis=1)

                ytd_period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
                if 'Target' in sheet_name:
                    paste_df = pd.concat([paste_df, pd.DataFrame({
                        'File Name': sheet_name,
                        'Entity': entity_col,
                        'Function': function_col,
                        'Account': account_col,
                        'Attribute Type': 'YTD Target',
                        'Period': ytd_period_value,
                        cost_type: sheet['YTD Total']
                    })], ignore_index=True)

                elif 'Fcst' in sheet_name:
                    paste_df = pd.concat([paste_df, pd.DataFrame({
                        'File Name': sheet_name,
                        'Entity': entity_col,
                        'Function': function_col,
                        'Account': account_col,
                        'Attribute Type': 'YTD Forecast',
                        'Period': ytd_period_value,
                        cost_type: sheet['YTD Total']
                    })], ignore_index=True)

                elif 'Actual' in sheet_name:
                    paste_df = pd.concat([paste_df, pd.DataFrame({
                        'File Name': sheet_name,
                        'Entity': entity_col,
                        'Function': function_col,
                        'Account': account_col,
                        'Attribute Type': 'YTD Actual',
                        'Period': ytd_period_value,
                        cost_type: sheet['YTD Total']
                    })], ignore_index=True)

            # 3. Prior Year FY
            prior_fy_cols = [col for col in sheet.columns if f'FY {current_year - 1}' in col]
            for prior_fy_col in prior_fy_cols:
                prior_fy_period_value = f"Dec-{str(current_year - 1)[-2:]}"
                paste_df = pd.concat([paste_df, pd.DataFrame({
                    'File Name': sheet_name,
                    'Entity': entity_col,
                    'Function': function_col,
                    'Account': account_col,
                    'Attribute Type': 'Prior Year FY',
                    'Period': prior_fy_period_value,
                    cost_type: sheet[prior_fy_col]
                })], ignore_index=True)

            # 4. Prior Year YTD
            prior_ytd_cols = [col for col in sheet.columns if f'YTD {current_year - 1}' in col]
            for prior_ytd_col in prior_ytd_cols:
                prior_ytd_period_value = f"{month_number_to_name(current_month)}-{str(current_year - 1)[-2:]}"
                paste_df = pd.concat([paste_df, pd.DataFrame({
                    'File Name': sheet_name,
                    'Entity': entity_col,
                    'Function': function_col,
                    'Account': account_col,
                    'Attribute Type': 'Prior Year YTD',
                    'Period': prior_ytd_period_value,
                    cost_type: sheet[prior_ytd_col]
                })], ignore_index=True)

    # Write the final DataFrame to an Excel file
    paste_df.to_excel('final_output_file.xlsx', sheet_name='Paste', index=False)

# Example usage
current_month = 6  # Example: June
current_year = 2024  # Example: Current year
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
process_paste_sheet(processed_file_path, current_month, current_year)
